## LangChain: Agents

### Outline:

> * Using **built** in LangChain tools: **DuckDuckGo search** and **Wikipedia**
> * Defining your **own tools**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os

api_key = os.getenv("GOOGLE_API_KEY")

In [5]:
# !pip install -U wikipedia

In [13]:
# !pip install langchain-experimental

In [9]:
# Chat model
from langchain_google_genai import ChatGoogleGenerativeAI

# Agents
from langchain_experimental.agents.agent_toolkits.python.base import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

# additional langchain tools
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.utilities import PythonREPL

In [24]:
chat_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

In [13]:
# !pip install numexpr

In [25]:
tools = load_tools(["llm-math","wikipedia"], llm=chat_model)

In [26]:
agent= initialize_agent(
    tools, 
    chat_model, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [27]:
agent.invoke("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Question: What is the 25% of 300?
Thought: I need to calculate 25% of 300. I can do this by multiplying 300 by 0.25.
Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 0.25"
}
```

Observation: Answer: 75.0
Thought:I now know the final answer
Final Answer: 75


> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75'}

---
### Wikipedia example

In [28]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"

agent.invoke(question) 



> Entering new AgentExecutor chain...
Question: Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?
Thought: I need to find out what book Tom M. Mitchell wrote. I will use Wikipedia to search for him and look for a book he authored.
Action:
```json
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of A

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Machine Learning'}

### Python Agent

In [29]:
agent = create_python_agent(
    chat_model,
    tool=PythonREPLTool(),
    verbose=True
)

In [32]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"]
                ]

In [33]:
agent.invoke(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
I need to sort a list of lists, where each inner list represents a customer with the first element being the last name and the second element being the first name. I need to sort first by last name and then by first name. I can use the sorted() function with a lambda function to achieve this.
Action: Python_REPL
Action Input:
```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too']]
sorted_customers = sorted(customers, key=lambda x: (x[0], x[1]))
print(sorted_customers)
```
Observation: [['Dolly', 'Too'], ['Harrison', 'Chase'], ['Lang', 'Chain']]

Thought:The customers are now sorted first by last name and then by first name.
Final Answer: [['Dolly', 'Too'], ['Harrison', 'Chase'], ['Lang', 'Chain']]


> Finished chain.


{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too']]",
 'output': "[['Dolly', 'Too'], ['Harrison', 'Chase'], ['Lang', 'Chain']]"}

### Define your own tool

In [36]:
from langchain.agents import tool, initialize_agent
from datetime import date

In [43]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [44]:
agent= initialize_agent(
    tools + [time], 
    chat_model, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [45]:
try:
    agent.invoke("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I need to use the time tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2025-01-09
Thought:I now know the final answer
Final Answer: 2025-01-09


> Finished chain.
